In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd

/opt/homebrew/anaconda3/envs/MLCS6140/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/anaconda3/envs/MLCS6140/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/homebrew/anaconda3/envs/MLCS6140/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <E459C462-F863-3A5A-AC9F-FD77B14BE845> /opt/homebrew/anaconda3/envs/MLCS6140/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
'''
    CODE TO PREPARE DATA.
'''
train_path = '/Users/rjaditya/Documents/projects/Kaggle/digit/dataset/train.csv'
test_path = '/Users/rjaditya/Documents/projects/Kaggle/digit/dataset/test.csv'

dataset = pd.read_csv(train_path, header=0)
dataset.astype(float)
submission_dataset = pd.read_csv(test_path, header=0)

y = dataset['label']
X = dataset.drop(labels='label', axis=1)


In [5]:
x_train, x_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)

# train_data = x_train
# train_data['label'] = y_train

# test_data = x_test
# test_data['label'] = y_test

# train_dataset = torch.tensor(train_data.values)
# test_dataset = torch.tensor(test_data.values)

x_train = torch.tensor(x_train.values)
y_train = torch.tensor(y_train.values)
x_test = torch.tensor(x_test.values)
y_test = torch.tensor(y_test.values)

training_set = torch.utils.data.TensorDataset(x_train, y_train)
validation_set = torch.utils.data.TensorDataset(x_test, y_test)

trainloader = torch.utils.data.DataLoader(training_set,
                                          batch_size=4,
                                          shuffle=False, num_workers=2)
validationloader = torch.utils.data.DataLoader(validation_set,
                                         shuffle=False, num_workers=2)



In [10]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.l1 = nn.Linear(784, 500)
        self.l2 = nn.Linear(500, 250)
        self.l3 = nn.Linear(250, 10)
    def forward(self, x):
        x = self.l1(x)
        x = nn.ReLU(x)
        x = self.l2(x)
        x = nn.ReLU(x)
        x = self.l3(x)
        return x

model = net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
loss = nn.BCELoss

In [7]:
print(trainloader)
for i, data in enumerate(trainloader):
    print(data)
    break

[tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), tensor([3, 9, 4, 5])]


In [11]:
training_loss = []
from tqdm import tqdm
for epoch in tqdm(range(200)):  # loop over the dataset multiple times
    running_loss = 0
    for i, data in enumerate(trainloader):
        x, y = data
        print(x)
        print(y)
        optimizer.zero_grad()
        output = model(x)
        print(output)
        _, y_hat = torch.max(output.data, 1)
        print(y_hat)
        tloss = loss(y_hat, y)
        tloss.backward()
        optimizer.step()
        running_loss += tloss.item()
    training_loss.append(running_loss)

  0%|          | 0/200 [00:00<?, ?it/s]

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([3, 9, 4, 5])


  0%|          | 0/200 [00:01<?, ?it/s]


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float